# Opencv项目实战-图像分割思路：使用otsu方法在HSV颜色空间完成预分割，将结果作为前背景的可能的标注，再使用grabcut方法完成分割

In [1]:
#coding:utf8
import os
import sys

import matplotlib.pyplot as plt
import matplotlib.image as mplimage
import numpy as np
import cv2

import math

class ImageSeg:
    def __init__(self,color_space='GRAY'):
        print ("init")
        self.color_space = color_space
        
    def otsu_seg(self,rgbimg):
        if self.color_space == 'GRAY':
            greyimg = cv2.cvtColor(rgbimg, cv2.COLOR_BGR2GRAY)
            greyblurimg = cv2.GaussianBlur(greyimg,(5,5),0)
            self.th,self.otsumask = cv2.threshold(greyblurimg,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        elif self.color_space == 'HSV':
            hsvimg = cv2.cvtColor(rgbimg,cv2.COLOR_BGR2HSV)
            greyimg = hsvimg[:,:,2]
            greyblurimg = cv2.GaussianBlur(greyimg,(5,5),0)
            self.th,self.otsumask = cv2.threshold(greyblurimg,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 
        elif self.color_space == 'Lab':
            labimg = cv2.cvtColor(rgbimg,cv2.COLOR_BGR2Lab)
            greyimg = labimg[:,:,1]
            greyblurimg = cv2.GaussianBlur(greyimg,(5,5),0)
            self.th,self.otsumask = cv2.threshold(greyblurimg,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU) 

    def grabcut(self,image):
        bgdModel = np.zeros((1,65),np.float64)
        fgdModel = np.zeros((1,65),np.float64)
        rect = (100,100,500,400)
        mask = np.zeros(image.shape[:2],np.uint8)
        mask[self.otsumask == 0] = cv2.GC_PR_BGD
        mask[self.otsumask == 255] = cv2.GC_PR_FGD
        mask, bgdModel, fgdModel = cv2.grabCut(image,mask,None,bgdModel,fgdModel,5,cv2.GC_INIT_WITH_MASK)
        self.grabcutmask = np.where((mask==2)|(mask==0),0,1).astype('uint8')*255
    #image = image*mask[:,:,np.newaxis]

    def process(self,image):
        ##1,获得二值分割结果
        height,width,channel = image.shape   
        self.otsu_seg(image)
        self.grabcut(image)
        return self.otsumask,self.grabcutmask

#部分结果如下：
 
 
#思考：如果前景，背景是反的，怎么办？

In [7]:
myimageseg = ImageSeg('HSV')
images = os.listdir('./flower/')
for imagename in images:
    if imagename.endswith('.jpg'):
        imagepath = os.path.join('./flower/',imagename)
        image = cv2.imread(imagepath)
        print ("processing:",imagepath)
        otsuresult,grabcutresult = myimageseg.process(image)
        cv2.imwrite(imagepath.replace('.jpg','_hsv_otsu.png'),otsuresult)
        cv2.imwrite(imagepath.replace('.jpg','_hsv_grab.png'),grabcutresult)
        cv2.namedWindow("image",0)
        cv2.imshow("image",image)
        cv2.namedWindow("otsu",0)
        cv2.imshow("otsu",otsuresult)
        cv2.namedWindow("grab",0)
        cv2.imshow("grab",grabcutresult)
        k = cv2.waitKey(10)
        if k == ord('q'):
            break

init
processing: ./flower/600.jpg
processing: ./flower/601.jpg
processing: ./flower/602.jpg
processing: ./flower/603.jpg
processing: ./flower/604.jpg
processing: ./flower/605.jpg
processing: ./flower/606.jpg
processing: ./flower/607.jpg
processing: ./flower/608.jpg
processing: ./flower/609.jpg
processing: ./flower/610.jpg
processing: ./flower/611.jpg
processing: ./flower/612.jpg
processing: ./flower/613.jpg
processing: ./flower/614.jpg
processing: ./flower/615.jpg
processing: ./flower/616.jpg
processing: ./flower/617.jpg
processing: ./flower/618.jpg
processing: ./flower/619.jpg
processing: ./flower/620.jpg
processing: ./flower/621.jpg
processing: ./flower/622.jpg
processing: ./flower/623.jpg
processing: ./flower/624.jpg
processing: ./flower/625.jpg
processing: ./flower/626.jpg
processing: ./flower/627.jpg
processing: ./flower/628.jpg
processing: ./flower/629.jpg
processing: ./flower/630.jpg
processing: ./flower/631.jpg
processing: ./flower/632.jpg
processing: ./flower/633.jpg
processin